In [1]:
import json

droid3d_anno_path = "droid"

with open(droid3d_anno_path + "/cam2base_extrinsics.json") as f:
    cam2base_extrinsics = json.load(f)

with open(droid3d_anno_path + "/cam2base_extrinsic_superset.json") as f:
    cam2base_extrinsic_superset = json.load(f)

with open(droid3d_anno_path + "/droid_language_annotations.json") as f:
    droid_language_annotations = json.load(f)

print(len(cam2base_extrinsic_superset))

24044


In [2]:
from utils import xyz_from_uvd, uvd_from_xyz, T_from_xyzrpy, xyzrpy_from_T

import glob
import json

droid_base_path = "/mnt/20T/droid_raw_1.0.1"

episode_meta_paths = glob.glob("*/success/*/*/metadata_*.json", root_dir=droid_base_path)
print(len(episode_meta_paths))

def wo_prefix(p):
    return "/".join(p.split("/")[3:])

for episode_meta_path in episode_meta_paths:
    with open(droid_base_path + "/" + episode_meta_path, "r") as f:
        metadata = json.load(f)
        
    uuid = metadata["uuid"]
    
    if uuid not in cam2base_extrinsic_superset:
        print(f"uuid {uuid} not in cam2base_extrinsic_superset")
        continue
    
    prefix_path = "/".join(episode_meta_path.split("/")[:-1])
    
    print(uuid)
    print(prefix_path)
    print(metadata)

    break

59740
uuid IPRL+7790ec0a+2023-06-27-20h-57m-09s not in cam2base_extrinsic_superset
uuid IPRL+7790ec0a+2023-06-27-20h-24m-51s not in cam2base_extrinsic_superset
IPRL+7790ec0a+2023-06-27-21h-02m-21s
IPRL/success/2023-06-27/Tue_Jun_27_21:02:21_2023
{'uuid': 'IPRL+7790ec0a+2023-06-27-21h-02m-21s', 'lab': 'IPRL', 'user': 'Marion Lepert', 'user_id': '7790ec0a', 'date': '2023-06-27', 'timestamp': '2023-06-27-21h-02m-21s', 'hdf5_path': 'success/2023-06-27/Tue_Jun_27_21:02:21_2023/trajectory.h5', 'building': '775 Roble', 'scene_id': 8351259745, 'success': True, 'robot_serial': 'panda-295341-1325480', 'r2d2_version': '1.3', 'current_task': 'Open or close hinged object (ex: hinged door, microwave, oven, book, dryer, toilet, box)', 'trajectory_length': 153, 'wrist_cam_serial': '12391924', 'ext1_cam_serial': '21582473', 'ext2_cam_serial': '28221883', 'wrist_cam_extrinsics': [0.32085092844058694, 0.061907514889166, 0.4160269019638068, 2.780198025634674, -0.21272213182917477, 1.7303129617793613], 'ex

In [3]:
import pytransform3d.transformations as pt
import numpy as np

Tbase2cam = pt.invert_transform(T_from_xyzrpy(cam2base_extrinsic_superset[uuid][metadata["ext1_cam_serial"]]))

Teef2tip = np.array([
    [0, 0, 1, 0],
    [0, -1, 0, 0],
    [1, 0, 0, 0.150],
    [0, 0, 0, 1]
])

In [4]:
import h5py

cartesian_positions = None

with h5py.File(droid_base_path + "/" + prefix_path + "/" + wo_prefix(metadata["hdf5_path"]), "r") as f:
    cartesian_positions = np.array(f["observation"]["robot_state"]["cartesian_position"])

In [5]:
import cv2
import numpy as np
import pyzed.sl as sl

def load_svo(svo_path, frame_idx=None):
    # Create a ZED camera object
    zed = sl.Camera()
    
    # Set configuration parameters
    init_params = sl.InitParameters()
    init_params.set_from_svo_file(svo_path)
    
    # Open the camera
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        print(f"Error {err}: Failed to open SVO file")
        return
    
    # Get intrinsics
    left_cam_calibration_params = zed.get_camera_information().camera_configuration.calibration_parameters.left_cam
    fx = left_cam_calibration_params.fx
    fy = left_cam_calibration_params.fy
    cx = left_cam_calibration_params.cx
    cy = left_cam_calibration_params.cy
    intrinsics = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    depth_scale = 1000
    
    # Create image
    image = sl.Mat()
    depth = sl.Mat()
    point_cloud = sl.Mat()
    
    # Read frames
    i = 0
    while zed.grab() == sl.ERROR_CODE.SUCCESS:
        zed.retrieve_image(image, sl.VIEW.LEFT) # Retrieve image
        zed.retrieve_measure(depth, sl.MEASURE.DEPTH) # Retrieve depth
        zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA)
        
        # zed.retrieve_image(image_r, sl.VIEW.RIGHT) # Retrieve right image
        # zed.retrieve_measure(depth_r, sl.MEASURE.DEPTH_RIGHT) # Retrieve right depth
        # zed.retrieve_measure(point_cloud_r, sl.MEASURE.XYZRGBA_RIGHT)

        # Convert to numpy array
        image_array = image.get_data().copy() # 不加copy好像会导致内存泄漏
        depth_array = depth.get_data().copy()
        point_cloud_array = point_cloud.get_data().copy()
        
        a = yield image_array, depth_array, point_cloud_array, intrinsics, depth_scale
        
        if frame_idx is not None and i == frame_idx:
            break
        
        i += 1
    
    # Close the camera
    zed.close()
    
    return

In [6]:
g = load_svo(droid_base_path + "/" + prefix_path + "/" + wo_prefix(metadata["ext1_svo_path"]), 0)

for idx, (image_array, depth_array, point_cloud_array, intrinsics, depth_scale) in enumerate(g):
    break

XYZRPYeef2cams = []
UVDeef2cams = []
for idx in range(len(cartesian_positions)):
    Ttip2base = T_from_xyzrpy(cartesian_positions[idx])
    Ttip2cam = Tbase2cam @ Ttip2base
    Teef2cam = Ttip2cam @ Teef2tip
    xyzrpy = xyzrpy_from_T(Teef2cam)
    XYZRPYeef2cams.append(xyzrpy)
    UVDeef2cams.append(uvd_from_xyz(xyzrpy[:3], intrinsics, depth_scale))

XYZRPYeef2cams = np.array(XYZRPYeef2cams)
UVDeef2cams = np.array(UVDeef2cams)

[2025-06-18 17:16:55 UTC][ZED][INFO] Logging level INFO
[2025-06-18 17:16:55 UTC][ZED][INFO] Logging level INFO
[2025-06-18 17:16:55 UTC][ZED][INFO] Logging level INFO
[2025-06-18 17:16:55 UTC][ZED][INFO] [Init]  Depth mode: PERFORMANCE
[2025-06-18 17:16:55 UTC][ZED][INFO] [Init]  Serial Number: S/N 21582473


In [7]:
image_array = cv2.cvtColor(np.asarray(image_array), cv2.COLOR_BGRA2BGR)
depth_array = depth_array.astype(np.uint16)
print(image_array.shape, image_array.dtype)
print(depth_array.shape, depth_array.dtype)

(720, 1280, 3) uint8
(720, 1280) uint16


/tmp/ipykernel_2913909/541270744.py:2: RuntimeWarning: invalid value encountered in cast
  depth_array = depth_array.astype(np.uint16)


In [8]:
output_dir = "output"
save_dir = output_dir + "/" + uuid
import os
os.makedirs(save_dir, exist_ok=True)

In [9]:
# print(metadata["current_task"])
print(droid_language_annotations[uuid]['language_instruction1'])
# print(list(droid_language_annotations.items())[1])

Open the door.


In [10]:
instruction = droid_language_annotations[uuid]['language_instruction1']

In [11]:
with open(save_dir + "/" + "intrinsics.json", "w") as f:
    json.dump(intrinsics.tolist(), f)

cv2.imwrite(save_dir + "/" + "image.png", image_array)
cv2.imwrite(save_dir + "/" + "depth.png", depth_array)

with open(save_dir + "/" + "XYZRPYeef2cams.json", "w") as f:
    json.dump(XYZRPYeef2cams.tolist(), f)

with open(save_dir + "/" + "instruction.json", "w") as f:
    json.dump(instruction, f)

with open(save_dir + "/" + "UVDeef2cams.json", "w") as f:
    json.dump(UVDeef2cams.tolist(), f)
